# Creating a Deployment

This notebook walks through deploying `task_maistro` as a self-hosted LangGraph Platform service.

## Code structure

[The following information is required](https://docs.langchain.com/langsmith/application-structure) to create a LangGraph Platform deployment:

* A [LangGraph API Configuration file](https://docs.langchain.com/langsmith/application-structure#configuration-file) — `langgraph.json`
* The graph definition — `src/task_maistro.py`
* A file specifying dependencies — `pyproject.toml`
* Environment variables — `.env` or `docker-compose.yml`

All of these are already present in this repository.

## CLI

The [LangGraph CLI](https://docs.langchain.com/langsmith/cli) is a command-line interface for building and managing LangGraph Platform deployments.

In [1]:
%%capture --no-stderr
%pip install -U langgraph-cli

To create a [self-hosted deployment](https://docs.langchain.com/langsmith/self_hosted_data_plane), we follow a few steps.

### Build Docker Image for LangGraph Server

We first use the LangGraph CLI to create a Docker image for the [LangGraph Server](https://docs.langchain.com/langsmith/langgraph-server).

This packages the graph and its dependencies into a Docker image.

Ensure that [Docker](https://docs.docker.com/engine/install/) is installed and then run from the project root:

```
$ langgraph build -t task-maistro
```

### Set Up Redis and PostgreSQL

If you already have Redis and PostgreSQL running (e.g., locally or on other servers), you can run the LangGraph Server container [by itself](https://docs.langchain.com/langsmith/deploy-hybrid#running-the-application-locally):

```
docker run \
    --env-file .env \
    -p 8123:8000 \
    -e REDIS_URI="foo" \
    -e DATABASE_URI="bar" \
    -e LANGSMITH_API_KEY="baz" \
    task-maistro
```

Alternatively, use the provided `docker-compose.yml` to spin up all three containers at once:

* `langgraph-redis`: Redis image for streaming pub/sub.
* `langgraph-postgres`: Postgres image for persistent state and store.
* `langgraph-api`: The pre-built `task-maistro` image.

Set the required environment variables in your `.env` file (`OPENAI_API_KEY`, `LANGSMITH_API_KEY`), then launch:

```
$ docker compose --env-file .env up
```